In [2]:
import csv
import spacy
from spacy import lookups
from spacy import tokenizer
import re
import nltk
from nltk import wordpunct_tokenize
nlp = spacy.load("en_core_web_sm")

print(spacy.__version__)
print(nltk.__version__)

3.2.2
3.6.5


In [2]:
import re
def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = words
    step1 = text.replace("\" ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot").replace(" '", "'")
    step6 = step5.replace("' ", "'")
    return step6.strip()

In [3]:
class Word:
    def __init__(self, index, form, lemma, upos, head, deprl, deps, children, siblings):
        self.index = index
        self.form = form
        self.lemma = lemma
        self.upos = upos
        self.head = head
        self.deprl = deprl
        self.deps = deps
        self.children = children
        self.siblings = siblings
        
    def word_info(self):
        print('\t'.join([self.index, self.form, self.lemma, self.upos, self.head, self.deprl]))

In [4]:
#functions

#NON-HABITUAL RULES

to_precedes_be_list = []
aux_children_list = []
aux_siblings_list = []
verbal_auxilary_list = []
copular_verb_list = []
other_list = []
nonhabitual = []
habitual = []


def rule_info(rule, text):
    print("RULE: ", rule)
    print("TEXT: ", text.strip())

#Rule #1: be is preceded by "to" or "na"
def to_precedes_be(previous):
    if(previous.lemma == "to" or previous.lemma == "na"):
                #rule_info("This is not habitual because it is preceded by 'to' or 'na':", text)
                to_precedes_be_list.append(text.strip())
                nonhabitual.append(text.strip())
                return True
                
#Rule #2: be has a sister or a child with the upos or deps of aux
def aux_children(be):
    for child in be.children:
        if(child.upos == "AUX" and child.deprl == "aux"):
            #rule_info("This is not habitual because it has a child that is aux/aux:", text)
            aux_children_list.append(text.strip())
            nonhabitual.append(text.strip())
            return True
def aux_siblings(be):
    for sibling in be.siblings:
        if(sibling.upos == "AUX" and sibling.deprl == "aux" and sibling.lemma != "do"):
            #rule_info("This is not habitual because it has a sister that is aux/aux:", text)
            aux_siblings_list.append(text.strip())
            nonhabitual.append(text.strip())
            return True
            
#HABITUAL RULES
            
#Rule #1: be has a upos of aux, a deprl of aux and its head has a upos of VERB  
def verbal_auxilary(be, wordList):
    if(be.upos == "AUX" and be.deprl == "aux"):
        for word in wordList:
            if(word.index == be.head and word.upos == "VERB"):
                #rule_info("This instance is habitual because be is aux/aux with a verb as its head:", text)
                verbal_auxilary_list.append(text.strip())
                habitual.append(text.strip())
                return True
                
#Rule #2: be has a upos of VERB
def copular_verb(be):
    if(be.upos == "VERB"):
        #rule_info("This instance is habitual because be is a verb:", text)
        copular_verb_list.append(text.strip())
        habitual.append(text.strip())
        return True

#create word list function
def create_word_list(entry, index, wordList, text): #DELETE TEXT FROM THE ARGUMENT HERE
    for j, line in enumerate(entry, start=1):
        if j == index:
            #finds the previous line
            previousLine = entry[j-2]
            previousLine = previousLine.split()
            previous = Word(previousLine[0], previousLine[1], previousLine[2], previousLine[3], previousLine[6], previousLine[7], previousLine[8], children=[], siblings=[])
            if(wordList):
                wordList.pop()
            wordList.append(previous)
            
            #finds be line
            beLine = line.split()
            be = Word(beLine[0], beLine[1], beLine[2], beLine[3], beLine[6], beLine[7], beLine[8], children=[], siblings=[])
            wordList.append(be)
            
        else:
            currentLine = line.split()
            word = Word(currentLine[0], currentLine[1], currentLine[2], currentLine[3], currentLine[6], currentLine[7], currentLine[8], children=[], siblings=[])
            wordList.append(word)
    
    #finding sisters and children
    for k, word in enumerate(wordList):
        if(word.head == be.index):
            be.children.append(word)
        elif word.head == be.head and word.form.lower() != "be": #changed from "word.index != be.index"
            be.siblings.append(word)
        
    return [previous, be]
    

In [5]:
#gets the line containing "be"

filepath = "/Users/wiler/Documents/College/Junior Year - Summer/Oral History/output/instances/allBeInstances.conllu"
leadingSpaces = len("# text = ")
count = 0
entry = []
allBeInstances = []
end = False
cleared = False
pattern = "[.,\/#!$%\^&\*;:{}=\-_`~()—\u2026]+[Bb]e$|^[Bb]e[\u2026.,\/#!$%\^&\*;:{}=\-_`~()—]+"

with open(filepath, "r", encoding="utf_8") as testConllu: #test this with beLinesOnly
    testConllu = testConllu.readlines()
    
    #goes through each line in an entry
    for row in testConllu:
        if row.startswith("# newdoc_id") or row.startswith("# sent_id"):
            continue
        elif row.startswith("# text"):
            text = row[leadingSpaces:]
        elif(row[0].isnumeric()):
            entry.append(row)
            if(row == testConllu[-1]): #checks if we are currently at the last line
                end = True
        else:
            end = True
            
            
        #we have reached the end of an entry    
        if(end is True):
            wordList = []
            textTokens = []
            beIndex = []
            
            parsedText = nlp(text) #let me try without stripping
            for token in parsedText:
                textTokens.append(token.text)
            
            for i, word in enumerate(textTokens): #might have to change word so that it doesn't get confused with the class
                if word.lower() == "be" or re.search(pattern, word.lower()):
                    beIndex.append(i+1)
 
            for index in beIndex:
              
                #the function returns an array in the format [previous, be]
                
                temp = create_word_list(entry, index, wordList, text.strip())
                previous = temp[0]
                be = temp[1]
    
                
                allBeInstances.append(text.strip())
                
                #non-habitual rules
                if(to_precedes_be(previous)):
                    continue
                elif(aux_children(be)):
                    continue
                elif(aux_siblings(be)):
                #habitual rules
                    continue
                elif(verbal_auxilary(be, wordList)):
                    continue
                elif(copular_verb(be)):
                    continue
                #unclassified instances
                else:
                    other_list.append(text.strip())
                    continue        
            
            end = False
            entry.clear()
            
            
        



    
    

#remember to clear entry at the end

In [6]:
print("Number of Non-Habitual Instances: ", len(nonhabitual))
print("Number of Habitual Instances: ", len(habitual))
print("Number of Unclassified Instances: ", len(other_list))
print("All Be Instances: ", len(allBeInstances))

Number of Non-Habitual Instances:  10339
Number of Habitual Instances:  265
Number of Unclassified Instances:  675
All Be Instances:  11279


In [7]:
#writes the sentences in plain text

with open("./output/nonhabitual/to_precedes_be.txt", "w", encoding="utf=8") as file:
    for line in to_precedes_be_list:
        file.write(line + "\n")
        
with open("./output/nonhabitual/aux_children.txt", "w", encoding="utf_8") as file:
    for line in aux_children_list:
        file.write(line + "\n")

with open("./output/nonhabitual/aux_siblings.txt", "w", encoding="utf_8") as file:
    for line in aux_siblings_list:
        file.write(line + "\n")
        
with open("./output/habitual/verbal_auxilary.txt", "w", encoding="utf_8") as file:
    for line in verbal_auxilary_list:
        file.write(line + "\n")

with open("./output/habitual/copular_verb.txt", "w", encoding="utf_8") as file:
    for line in copular_verb_list:
        file.write(line + "\n")
        
with open("./output/habitual/all_habitual.txt", "w", encoding="utf_8") as file:
    for line in habitual:
        file.write(line + "\n")

with open("./output/nonhabitual/all_nonhabitual.txt", "w", encoding="utf_8") as file:
    for line in nonhabitual:
        file.write(line + "\n")

In [8]:
#writes the sentences in ConLLu format

parsed = [nlp(line) for line in to_precedes_be_list]
with open("./output/nonhabitual/to_precedes_be.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = to_precedes_be.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')
        
        
parsed = [nlp(line) for line in aux_children_list]
with open("./output/nonhabitual/aux_children.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = aux_children.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')
        
        
parsed = [nlp(line) for line in aux_siblings_list]
with open("./output/nonhabitual/aux_siblings.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = aux_siblings.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')
        
parsed = [nlp(line) for line in verbal_auxilary_list]        
with open("./output/habitual/verbal_auxilary.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = verbal_auxilary.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')

parsed = [nlp(line) for line in copular_verb_list]
with open("./output/habitual/copular_verb.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = copular_verb.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')
        
parsed = [nlp(line) for line in other_list]
with open("./output/nonhabitual/other_list.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = other_list.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')

Selects 10% of each category for random quality checking by annotators

In [9]:
import random

#to_precedes_be
sample_size = int(len(to_precedes_be_list) * .10)
to_precedes_be_list_random = random.sample(to_precedes_be_list, sample_size)

parsed = [nlp(line) for line in to_precedes_be_list_random]
with open("./output/quality_check/to_precedes_be_random.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = to_precedes_be_random.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')


#aux_children
sample_size = int(len(aux_children_list) * .10)
aux_children_list_random = random.sample(aux_children_list, sample_size)
print(len(aux_children_list_random))

parsed = [nlp(line) for line in aux_children_list_random]

with open("./output/quality_check/aux_children_random.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = aux_children_random.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')
        
#aux_siblings
sample_size = int(len(aux_siblings_list) * .10)
aux_siblings_list_random = random.sample(aux_siblings_list, sample_size)
print(len(aux_siblings_list_random))
parsed = [nlp(line) for line in aux_siblings_list_random]

with open("./output/quality_check/aux_siblings_random.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = aux_siblings_random.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')


#other_list
sample_size = int(len(other_list) * .10)
other_list_random = random.sample(other_list, sample_size)
print(len(other_list_random))
parsed = [nlp(line) for line in other_list_random]

with open("./output/quality_check/other_list_random.conllu", "w", encoding="utf=8") as output:
    for i, parsedSent in enumerate(parsed):
        output.write("# newdoc_id = other_list_random.txt" + '\n')
        output.write("# sent_id = " + str(i) + '\n')
        output.write("# text = " + str(parsedSent.text) + '\n')
        
        for j, token in enumerate(parsedSent):
            word_id = str(j+1) 
            wordform = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = '_'
            feats = '_'
            deps =  '_'
            misc = '_'
            if token.dep_== "ROOT":
                head = '0'
                deprel = 'root'
            else:
                head = str(token.head.i + 1)
                deprel = token.dep_.lower()
            writeStr = '\t'.join([word_id, wordform, lemma, upos, xpos, feats, head, deprel, deps, misc]) + '\n'
            output.write(writeStr)
        
        output.write('\n')
        
#copular_verb and verbal_auxilary do not have to be sampled due to their small quantity and importance; all instances will be checked for habituality and concordance to the rules

370
125
67


In [10]:
#puts everything into a csv file where the first column holds the text and the second column states whether the sentence followed the rule or not

other_list = []


#to_precedes_be
with open('./output/quality_check/to_precedes_be.csv', 'w', encoding="utf-8-sig") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Text', '1 or 0'])
    for sent in to_precedes_be_list_random:
        filewriter.writerow([sent])
        
#aux_children
with open('./output/quality_check/aux_children.csv', 'w', encoding="utf-8-sig") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Text', '1 or 0'])
    for sent in aux_children_list_random:
        filewriter.writerow([sent])
        
#aux_siblings
with open('./output/quality_check/aux_siblings.csv', 'w', encoding="utf-8-sig") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Text', '1 or 0'])
    for sent in aux_siblings_list_random:
        filewriter.writerow([sent])
        
#verbal_auxilary
with open('./output/quality_check/verbal_auxilary.csv', 'w', encoding="utf-8-sig") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Text', '1 or 0'])
    for sent in verbal_auxilary_list:
        filewriter.writerow([sent])
        
#copular_verb
with open('./output/quality_check/copular_verb.csv', 'w', encoding="utf-8-sig") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Text', '1 or 0'])
    for sent in copular_verb_list:
        filewriter.writerow([sent])

#other_list
with open('./output/quality_check/other.csv', 'w', encoding="utf-8-sig") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Text', '1 or 0'])
    for sent in other_list_random:
        filewriter.writerow([sent])